- Classify image similar to each other
- Based on n_cluster = 3 -> classify to 3 classes

In [10]:
import torch
import clip
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
import os
import shutil

In [13]:
# Thiết lập thiết bị và tải mô hình CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model, preprocess = clip.load("ViT-B/32", device=device)

def get_image_features(image_path):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    return image_features.cpu().numpy()

def cluster_images(image_paths, n_clusters=3):
    features = [get_image_features(image_path) for image_path in image_paths]
    features = np.vstack(features)
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(features)
    return kmeans.labels_

def create_directories(base_dir, n_clusters):
    for i in range(n_clusters):
        cluster_dir = os.path.join(base_dir, f"Cluster_{i}")
        if not os.path.exists(cluster_dir):
            os.makedirs(cluster_dir)

def move_images_to_clusters(image_paths, labels, base_dir):
    for image_path, label in zip(image_paths, labels):
        cluster_dir = os.path.join(base_dir, f"Cluster_{label}")
        shutil.move(image_path, cluster_dir)

Using device: cuda


In [ ]:
image_folder = "D:\\Code Space\\AI\\image_classification\\image\\categorize"
output_folder = "D:\\Code Space\\AI\\image_classification\\image\\output"
n_clusters = 3

image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]

# Nhóm các ảnh
labels, features = cluster_images(image_paths, n_clusters=n_clusters)

# Nhóm các ảnh
labels = cluster_images(image_paths, n_clusters=n_clusters)

# Tạo các thư mục cho các cụm
create_directories(output_folder, n_clusters)

# Di chuyển các ảnh vào các thư mục cụm tương ứng
move_images_to_clusters(image_paths, labels, output_folder)

print("Images have been grouped and moved to corresponding directories.")